В этой статье мы разберем способы обращения операции свертки на практике с использованием библиотеки TensorFlow.

На исследование этого вопроса меня натолкнула статья [Visualizing and Understanding Convolutional Networks]($Visualizing and Understanding Convolutional Networks$) (M. Zeiler, R. Fergus, 2013). В ней авторы используют транспонированную свертку с теми же весами, чтобы восстановить входные данные сверточного слоя по выходным данным. Возник вопрос: почему применяя транспонированную свертку мы надеемся восстановить входные данные? Чтение предыдущих статей по этой теме (обзоры на них: [1]($Deconvolutional Networks$), [2]($Adaptive Deconvolutional Networks for Mid and High Level Feature Learning$)) ответа на вопрос не дало. Это натолкнуло меня на то, чтобы провести ряд собственных экспериментов.

Мы сравним (в теории и на практике) три способа обращения операции свертки:
1. Аналитическое решение с помощью псевдообратной матрицы
2. Решение с помощью методов оптимизации (Adam и L-BFGS)
3. Транспонированная свертка

### Свертка как матричное умножение

Мы будем рассматривать дискретную свертку с несколькими фильтрами и входными каналами, которая используется в сверточных нейронных сетях. Функцию активации и прибавление bias'а мы не рассматриваем, так как они применяются поэлементно, и задача их обращения либо тривиальна, либо не имеет решения. Подробно про операцию свертки можно почитать в статье [A guide to convolution arithmetic for deep learning](https://arxiv.org/abs/1603.07285), анимированные изображения можно найти на [Github](https://github.com/vdumoulin/conv_arithmetic).

Свертка - это линейное отображение, то есть элементы выходного массива линейно зависят от элементов входного массива, а ноль отображается в ноль. Если мы зафиксируем размер входного массива, то мы можем "вытянуть в векторы" входной и выходной массивы. Для иллюстрации, в numpy это делается операцией `array.reshape(-1)` для одного  изображения или `array.reshape(len(array), -1)` для батча. Любое линейное отображение можно представить в виде умножения на матрицу (справа или слева, в зависимости от того, используем мы вектор-строку или вектор-столбец).

Это означает, что *свертка с фиксированным размером входного изображения может быть представлена как умножение на матрицу*. Большая часть элементов этой матрицы по определению будут нулевыми, поскольку элемент выходного массива связан не со всеми элементами входного массива, а лишь с теми, которые находятся в рамках скользящего окна. Кроме того некоторые пары элементов этой матрицы по определению будут равны, так как мы используем одни и те же фильтры в каждой позиции скользящего окна.

*Примечание: если в каждой позиции скользящего окна мы используем разные фильтры, то такая операция в нейронной сети называется Locally-Connected layer и была описана в [данной статье]($Emergence of Complex-Like Cells in a Temporal Product Network with Local Receptive Fields$).*

### Задача обращения операции свертки

Задача обращения свертки является обратной задачей: мы хотим восстановить $x$, зная $f(x)$. Поскольку операция свертки линейна, то мы имеем дело с [линейной обратной задачей](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%B0%D1%8F_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0#%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D0%BE%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%B0%D1%8F_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0).

**Корректность задачи**

Если решение обратной задачи существует и единственно (а также непрерывно зависит от исходных данных, что в случае свертки всегда верно), то такая обратная задача называется *корректно поставленной (well posed)*, в противном случае - *некорректно поставленной (ill-posed)*.

В задаче обращения свертки решение не всегда будет существовать и не всегда будет единственно. Самый простой пример - фильтр $f$, состоящий из нулей. Если $f(x)$ также состоит из нулей, то любое $x$ является решением, в противном случае решений нет. Рассмотрим теперь одномерную свертку с окном размера 3, с наличием или отсутствием padding'а:

<img src="assets/deconv4.jpg" width="900" align="center">

Для простоты размер входных данных максимально уменьшен.

В случае *without padding* мы имеем одно уравнение $y_1 = w_1x_1 + w_2x_2 + w_3x_3$ и три неизвестных, а значит есть много решений. Матрица преобразования будет прямоугольной и не будет иметь обратной матрицы. Но если мы доопределим  элементы по краям ($x_1$, $x_2$), то для внутренних элементов решение будет едитственным.

В случае *with padding* мы имеем три уравнения и три неизвестных. Представим свертку в виде матрицы (см. справа на иллюстрации). Решение будет единственным если матрица невырождена.

Из реальных примеров: операция обесцвечивания является сверткой с одним фильтром и тремя входными каналами. Обратная задача некорректно поставлена, то есть информация о цвете безвозвратно потеряна. Вообще во многих случаях обращение свертки имеет не единственное решение, поэтому точно ответ найти невозможно и остается лишь искать наиболее правдоподобный ответ, применяя регуляризацию (об этом далее).

**Обусловленность задачи**

Обратная задача называется *плохо обусловленной*, если небольшие изменения в исходных данных $f(x)$ могут привести к большим изменениям в ответе $x$. Изменения в $f(x)$ могут быть вызваны во-первых случайным шумом (например, если данные получены с измерительных приборов), а во-вторых погрешностями арифметики с плавающей запятой.

Например, рассмотрим операцию размытия дискретным [гауссовым фильтром](https://en.wikipedia.org/wiki/Gaussian_filter).

Фильтр и результат его применения показан на иллюстрации.